In [ ]:
# Configure plotting
%matplotlib inline

import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['figure.figsize'] = (10, 4)
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
matplotlib.rcParams['font.size'] = 16

import os
from pathlib import Path
from datetime import datetime

import GPy
import numpy as np
RANDOM_SEED=123
np.random.seed(RANDOM_SEED)

import seaborn as sns

from utils.model import GPyModel

from utils.utils import discretized_normal_distribution, sample_index_from_p
from utils.utils import w_x_t, kappa_x_w, MMD, worst_context_distribution_DRO


## Handcrafted function

In [ ]:
# For visualization purposes,
# the function surface we plot is denser than the function surface we optimize.
# We load both of them individually
load_path = os.path.join("functions", "custom.npy")
load_path_plot = os.path.join("functions", "custom_plot.npy")

# Domain for action and context
X_lim = [-3.5, 3.5]
C_lim = [-12, 12]

# Action and context cardinalities
N_x = 8
N_c = 50

X = np.linspace(X_lim[0], X_lim[1], N_x)  # Vector of X values
C = np.linspace(C_lim[0], C_lim[1], N_c)  # Vector of C values

# Generation of matrix M for MMD
C_kern = GPy.kern.RBF(input_dim=1, lengthscale=5)
M = C_kern.K(C.reshape(N_c, 1))

# Generation of all context, action pairs (input domain).
x, c = np.meshgrid(X, C)
CX_pairs = np.concatenate([x.reshape(-1, 1), c.reshape(-1, 1)], axis=1)  # All C, X pairs

# Objective function and its flattened version
func_mat_CX = np.load(load_path).reshape(N_c, N_x)
func_vals = func_mat_CX.reshape(-1, 1)


# ----- These are only for plotting of objective function
# Since we are not plotting the surface alongside the regrets, we're not using these.

N_x_p = 50

func_mat_CX_plot = np.load(load_path_plot).reshape(N_c, N_x_p)

# -----

fig, ax = plt.subplots(1, 2)

ax[0].imshow(func_mat_CX_plot, interpolation='bicubic')
ax[1].imshow(M, interpolation='bicubic')

In [ ]:
def create_w_true_and_ref():
    """Generates and returns w_true and w_ref according to the experiment setup."""
    
    w_ref = discretized_normal_distribution(C, 2, (1)**2)
    w_true = discretized_normal_distribution(C, 0, (5)**2)

    return w_true, w_ref

In [ ]:
def get_observation(flat_indices, noise_std=0):
    """Given a list of (action, context) indices, returns the noisy observations.
    Note that, instead of 2D indexing, flattened indices are used."""
    Y_sample = func_vals[flat_indices].reshape(-1, 1)
    Y_sample_n = Y_sample + np.random.randn(*Y_sample.shape) * noise_std
    
    return Y_sample_n

In [ ]:
def BO_loop_DRO(model, distributions, contexts, epsilon_coef, beta, T):
    CX_t_indices = []
    for t in range(T):
        # True and reference distributions
        w_true, w_ref = distributions[t]

        # The variable epsilon here is the radius of the ambiguity set of DRBO
        # true distance multiplied with a coefficient, e.g. 1/3
        epsilon_t = MMD(M, w_true, w_ref) * epsilon_coef

        # Modelling
        mu, var = model.predict(CX_pairs)
        CX_pair_UCB = mu + beta*np.sqrt(var)
        CX_mat_UCB = CX_pair_UCB.reshape(N_c, N_x)

        # For each action, calculate worst expected reward in the ambiguity set
        worst_expected_reward_x = np.zeros(N_x)
        for x_ind in range(N_x):
            UCB_x = CX_mat_UCB[:, x_ind]
            worst_w_x = worst_context_distribution_DRO(M, w_ref, UCB_x, epsilon_t)
            worst_expected_reward_x[x_ind] = np.dot(worst_w_x.T, UCB_x)
        
        # Choose maximum of these worst expected rewards.
        chosen_action_index = np.argmax(worst_expected_reward_x)
        # Context is observed from precomputed context vector.
        observed_context_index = contexts[t]
        # Add selected action and observed context indices to the array to be returned
        CX_t_indices.append([observed_context_index, chosen_action_index])

        # Add new sample to model and update the model
        cx_flattened_index = observed_context_index*N_x + chosen_action_index
        CX_sample = CX_pairs[[cx_flattened_index]]
        Y_sample_n = get_observation([cx_flattened_index], noise_std=model.noise_std)
        model.add_sample(CX_sample, Y_sample_n)
        model.update()

        print("Chosen action point:", CX_sample[0][0])

    return CX_t_indices

In [ ]:
def BO_loop_RS(model, distributions, contexts, beta, T, taus):
    # Aspiration is currently not used.

    kappas = np.zeros(T)
    CX_t_indices = []
    for t in range(T):
        w_true, w_ref = distributions[t]
        w_true = w_true.reshape(N_c, 1)
        w_ref = w_ref.reshape(N_c, 1)
        
        tau = taus[t]
        
        # Modelling
        mu, var = model.predict(CX_pairs)
        CX_pair_UCB = mu + beta*np.sqrt(var)
        CX_mat_UCB = CX_pair_UCB.reshape(N_c, N_x)

        # For each action, calculate kappa_hat_tau,t and kappa_tau,t
        list_kappa_x = np.zeros(N_x)
        list_kappa_hat_x = np.zeros(N_x)
        for x_ind in range(N_x):
            ucb_x = CX_mat_UCB[:, x_ind].reshape(-1, 1)
            w_bar_x_t = w_x_t(M, w_ref, ucb_x, tau)
            kappa_hat_x = kappa_x_w(M, w_ref, w_bar_x_t, ucb_x, tau, clip=False)
            list_kappa_hat_x[x_ind] = kappa_hat_x
            
            f_x = func_mat_CX[:, x_ind].reshape(-1, 1)
            w_dbar_x_t = w_x_t(M, w_ref, f_x, tau)
            kappa_x = kappa_x_w(M, w_ref, w_dbar_x_t, f_x, tau, clip=False)
            list_kappa_x[x_ind] = kappa_x

        # Choose action with minimum kappa_hat.
        chosen_action_index = np.argmin(list_kappa_hat_x)
        # Context is observed from precomputed context vector.
        observed_context_index = contexts[t]
        # Add selected action and observed context indices to the array to be returned
        CX_t_indices.append([observed_context_index, chosen_action_index])

        # Add new sample to model and update the model
        cx_flattened_index = observed_context_index*N_x + chosen_action_index
        CX_sample = CX_pairs[[cx_flattened_index]]
        Y_sample_n = get_observation([cx_flattened_index], noise_std=model.noise_std)
        model.add_sample(CX_sample, Y_sample_n)
        model.update()

        print("Chosen action point:", CX_sample[0][0], " - Kappa:", np.min(list_kappa_hat_x))

        # Save minimum kappa for regret calculation.
        min_kappa = np.min(list_kappa_x)
        kappas[t] = min_kappa

    return CX_t_indices, kappas

In [ ]:
# Loop over tau's and save all experiments


# Model simulation parameters: simulation count, timestep count and dro epsilons

# Reset the randomness
np.random.seed(RANDOM_SEED)

sim_count = 10
timestep_count = 200 + 1  # +1 for undersampled plotting (last tick in the plot)
initial_sample_cnt = 3

# Different ball radius coefficients for DRO
RS_aspirations = [0.9]  # Currently not used

# Modelling parameters
input_dim = 2
output_dim = 1
noise_std = 0.02
noise_var = np.square(noise_std)
beta = 2

ker = GPy.kern.RBF(input_dim=input_dim, ARD=True)
ker.lengthscale = (0.2, 5)


simulation_name = datetime.now().strftime("%m_%d_%Y-%H_%M_%S")
simulation_folder = os.path.join(
    "results",
    f"synth_TAU_sim{sim_count}_ts{timestep_count}-std-{noise_std:.2f}-{simulation_name}"
)
os.makedirs(simulation_folder, exist_ok=False)


# tau in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.]
fixed_tau_list = np.linspace(0.1, 1, 10).round(2)
for fixed_tau in fixed_tau_list:
    # Reset the randomness
    np.random.seed(RANDOM_SEED)
    
    # Model simulations

    # Initial samples for each simulation
    initial_samples_sims = np.empty((sim_count, initial_sample_cnt, input_dim+output_dim))
    # Time dependent true and reference distributions for each simulation
    distributions_w_true_and_ref_sims = np.empty((sim_count, timestep_count, 2, N_c))
    # Chosen (action, context) pairs for each simulation of RS
    indices_RS_sims_asp = np.empty((len(RS_aspirations), sim_count, timestep_count, 2))
    # Kappa prime values for each simulation and timestep, we calculate these in RS loop
    kappas_sims_asp = np.empty((len(RS_aspirations), sim_count, timestep_count))
    # Tau values for each simulation and timestep, since we use a fixed tau this is constant
    taus_sims_asp = np.empty((len(RS_aspirations), sim_count, timestep_count))
    if fixed_tau is not None:
        taus_sims_asp[:] = fixed_tau

    for sim in range(sim_count):
        print(f"\n\n------------------------ SIMULATION NUMBER: {sim:02d} ------------------------\n\n")

        # Randomly select initial samples
        cx_flattened_initial_indices = np.random.randint(
            low=0, high=N_c*N_x, size=initial_sample_cnt,
        )
        initial_CX = CX_pairs[cx_flattened_initial_indices]
        initial_Y_n = get_observation(cx_flattened_initial_indices)
        initial_samples_sims[sim] = np.hstack((initial_CX, initial_Y_n))


        # Create context distributions and sample context at each timestep for all algorithms.
        # All algorithms run with same distributions and contexts for each simulation.
        context_counts_wrt_true = np.zeros((N_c, 1))
        sim_contexts = np.empty(timestep_count, dtype=int)
        for t in range(timestep_count):
            w_true, w_ref = create_w_true_and_ref()
            distributions_w_true_and_ref_sims[sim, t, 0] = w_true.flatten()
            distributions_w_true_and_ref_sims[sim, t, 1] = w_ref.flatten()
            ts_context = sample_index_from_p(w_true)
            
            sim_contexts[t] = ts_context

            if fixed_tau is None:
                pass

        # RS simulation
        for asp_i, asp in enumerate(RS_aspirations):
            print(f"\nRS with aspiration={asp:.3f}\n")
            model_RS = GPyModel(input_dim, output_dim, noise_var, ker=ker)

            model_RS.add_sample(initial_CX, initial_Y_n)
            model_RS.update()
            
            chosen_indices_RS, kappas = BO_loop_RS(
                model_RS, distributions=distributions_w_true_and_ref_sims[sim],
                contexts=sim_contexts, beta=beta, T=timestep_count, taus=taus_sims_asp[asp_i, sim]
            )
            indices_RS_sims_asp[asp_i][sim] = chosen_indices_RS
            kappas_sims_asp[asp_i][sim] = kappas


    # Write simulation results
    sub_simulation_folder = os.path.join(
        simulation_folder,
        f"tau_{fixed_tau}"
    )
    os.makedirs(sub_simulation_folder, exist_ok=False)

    np.save(os.path.join(sub_simulation_folder, "initial_samples_sims"), initial_samples_sims)
    np.save(os.path.join(sub_simulation_folder, "distributions_w_true_and_ref_sims"), distributions_w_true_and_ref_sims)
    np.save(os.path.join(sub_simulation_folder, "indices_RS_sims_asp"), indices_RS_sims_asp)
    np.save(os.path.join(sub_simulation_folder, "kappas_sims_asp"), kappas_sims_asp)
    np.save(os.path.join(sub_simulation_folder, "taus_sims_asp"), taus_sims_asp)


In [ ]:
# Loop over tau's and save all experiments


# Model simulation parameters: simulation count, timestep count and dro epsilons

# Reset the randomness
np.random.seed(RANDOM_SEED)

sim_count = 10
timestep_count = 200 + 1  # +1 for undersampled plotting (last label to show)
initial_sample_cnt = 3

# Different ball radius coefficients for DRO
DRO_epsilons = np.logspace(-1, 1, 11, base=10)
RS_aspirations = [0.9]  # Currently not used

# Time independent tau value
fixed_tau = 0.6
assert fixed_tau is None or (fixed_tau is not None and len(RS_aspirations)==1)

# Modelling parameters
input_dim = 2
output_dim = 1
noise_std = 0.02
noise_var = np.square(noise_std)
beta = 2

ker = GPy.kern.RBF(input_dim=input_dim, ARD=True)
ker.lengthscale = (0.2, 5)


simulation_folder = os.path.join(
    "results",
    f"synth_TAU_sim{sim_count}_ts{timestep_count}-std-{noise_std:.2f}-{simulation_name}"
)
# os.makedirs(simulation_folder, exist_ok=False)


# Reset the randomness
np.random.seed(RANDOM_SEED)

# Model simulations

# Initial samples for each simulation
initial_samples_sims = np.empty((sim_count, initial_sample_cnt, input_dim+output_dim))
# Time dependent true and reference distributions for each simulation
distributions_w_true_and_ref_sims = np.empty((sim_count, timestep_count, 2, N_c))
# Chosen (action, context) pairs for each simulation of each DRO epsilon
indices_DRO_sims_eps = np.empty((len(DRO_epsilons), sim_count, timestep_count, 2))
# Tau values for each simulation and timestep, since we use a fixed tau this is constant
taus_sims_asp = np.empty((len(RS_aspirations), sim_count, timestep_count))
if fixed_tau is not None:
    taus_sims_asp[:] = fixed_tau

for sim in range(sim_count):
    print(f"\n\n------------------------ SIMULATION NUMBER: {sim:02d} ------------------------\n\n")

    # Randomly select initial samples
    cx_flattened_initial_indices = np.random.randint(
        low=0, high=N_c*N_x, size=initial_sample_cnt,
    )
    initial_CX = CX_pairs[cx_flattened_initial_indices]
    initial_Y_n = get_observation(cx_flattened_initial_indices)
    initial_samples_sims[sim] = np.hstack((initial_CX, initial_Y_n))


    # Create context distributions and sample context at each timestep for all algorithms.
    # All algorithms run with same distributions and contexts for each simulation.
    context_counts_wrt_true = np.zeros((N_c, 1))
    sim_contexts = np.empty(timestep_count, dtype=int)
    for t in range(timestep_count):
        w_true, w_ref = create_w_true_and_ref()
        distributions_w_true_and_ref_sims[sim, t, 0] = w_true.flatten()
        distributions_w_true_and_ref_sims[sim, t, 1] = w_ref.flatten()
        ts_context = sample_index_from_p(w_true)
        
        sim_contexts[t] = ts_context

        if fixed_tau is None:
            pass
        
    # DRO simulations with different epsilons
    for eps_i, eps in enumerate(DRO_epsilons):
        print(f"\nDRO with eps={eps:.3f}\n")
        model_DRO = GPyModel(input_dim, output_dim, noise_var, ker=ker)

        model_DRO.add_sample(initial_CX, initial_Y_n)
        model_DRO.update()
        
        chosen_indices_DRO = BO_loop_DRO(
            model_DRO, distributions=distributions_w_true_and_ref_sims[sim], contexts=sim_contexts,
            epsilon_coef=eps, beta=beta, T=timestep_count
        )
        indices_DRO_sims_eps[eps_i][sim] = chosen_indices_DRO


# Write simulation results
sub_simulation_folder = os.path.join(
    simulation_folder,
    f"eps"
)
os.makedirs(sub_simulation_folder, exist_ok=False)

np.save(os.path.join(sub_simulation_folder, "initial_samples_sims"), initial_samples_sims)
np.save(os.path.join(sub_simulation_folder, "distributions_w_true_and_ref_sims"), distributions_w_true_and_ref_sims)
np.save(os.path.join(sub_simulation_folder, "indices_DRO_sims_eps"), indices_DRO_sims_eps)
np.save(os.path.join(sub_simulation_folder, "taus_sims_asp"), taus_sims_asp)


In [ ]:
# Plot palettes
colors = sns.color_palette("dark")
greens = sns.color_palette("BuGn", 10)
blues = sns.color_palette("PuBu", 10)
reds = sns.color_palette("YlOrRd", 10)

In [ ]:
# Read simulation results


# [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]
fixed_tau_list = np.linspace(0.1, 1, 10).round(2)

cum_rewards = np.empty((1, len(fixed_tau_list)))
for tau_i, fixed_tau in enumerate(fixed_tau_list):
    distributions_w_true_and_ref_sims = np.load(
        os.path.join(simulation_folder, f"tau_{fixed_tau}", "distributions_w_true_and_ref_sims.npy")
    )
    indices_RS_sims_asp = np.load(
        os.path.join(simulation_folder, f"tau_{fixed_tau}", "indices_RS_sims_asp.npy")).astype(int
    )

    # Expected rewards of simulations and timesteps w.r.t. f
    expected_reward_true_sims_ts = distributions_w_true_and_ref_sims[:, :, 0, :] @ func_mat_CX

    # MMD distances of simulations and timesteps
    mmd_sims_ts = np.empty((sim_count, timestep_count))
    for sim_i in range(sim_count):
        for ts_i in range(timestep_count):
            mmd_sims_ts[sim_i, ts_i] = MMD(M, *distributions_w_true_and_ref_sims[sim_i, ts_i])

    # Calculate rewards

    # RS rewards
    true_rewards_of_actions_RS_asp = np.empty((len(RS_aspirations), sim_count, timestep_count))
    for a_i in range(len(RS_aspirations)):
        true_rewards_of_actions_RS_asp[a_i] = expected_reward_true_sims_ts[
            np.arange(sim_count)[:, None],
            np.arange(timestep_count),
            indices_RS_sims_asp[a_i, :, :, 1].astype(int)
        ]

    cum_rewards[0][tau_i] = true_rewards_of_actions_RS_asp[0].mean(axis=0).mean()

DRO_epsilons = np.logspace(-1, 1, 11, base=10)
cum_rewards_dro = np.empty((1, len(DRO_epsilons)))
for eps_i, eps in enumerate(DRO_epsilons):
    distributions_w_true_and_ref_sims = np.load(
        os.path.join(simulation_folder, f"eps", "distributions_w_true_and_ref_sims.npy")
    )
    indices_DRO_sims_eps = np.load(
        os.path.join(simulation_folder, f"eps", "indices_DRO_sims_eps.npy")).astype(int
    )

    # Expected rewards of simulations and timesteps w.r.t. f
    expected_reward_true_sims_ts = distributions_w_true_and_ref_sims[:, :, 0, :] @ func_mat_CX

    # MMD distances of simulations and timesteps
    mmd_sims_ts = np.empty((sim_count, timestep_count))
    for sim_i in range(sim_count):
        for ts_i in range(timestep_count):
            mmd_sims_ts[sim_i, ts_i] = MMD(M, *distributions_w_true_and_ref_sims[sim_i, ts_i])

    # Calculate rewards

    # DRO rewards
    true_rewards_of_actions_DRO_eps = np.empty((len(DRO_epsilons), sim_count, timestep_count))
    for e_i in range(len(DRO_epsilons)):
        true_rewards_of_actions_DRO_eps[e_i] = expected_reward_true_sims_ts[
            np.arange(sim_count)[:, None],
            np.arange(timestep_count),
            indices_DRO_sims_eps[e_i, :, :, 1].astype(int)
        ]

    cum_rewards_dro[0][eps_i] = true_rewards_of_actions_DRO_eps[eps_i].mean(axis=0).mean()


matplotlib.rcParams['figure.figsize'] = (6, 4)
fig, ax = plt.subplots(nrows=1, ncols=1)

# Label, color, linestyle
lcls = [
    ("RoBOS", reds[-1], "-", "."),
    ("DRBO", blues[-2], ":", "."),
]

ax.set_xlabel(r"$\tau$")
ax.set_ylabel(r"Avg. Reward")

label, color, linestyle, marker = lcls[0]
ax.plot(
    fixed_tau_list, cum_rewards[0], label=label, c=color, linestyle=linestyle, marker=marker
)
label, color, linestyle, marker = lcls[1]
ax.plot(
    np.linspace(0.1, 1, 11).round(2), cum_rewards_dro[0], label=label, c=color, linestyle=linestyle, marker=marker
)

ax.set_xticks(np.linspace(0.1, 1, 10).round(2))
ax.set_xticklabels(np.linspace(0.1, 1, 10).round(2))

secax0 = ax.secondary_xaxis("top")
secax0.set_xlabel(r"$\times\epsilon$")
secax0.set_xticks(np.linspace(0.1, 1, 11).round(2))
secax0.set_xticklabels(DRO_epsilons.round(2), fontsize=10)


ax.legend()
plt.tight_layout()
plt.savefig(os.path.join(Path(simulation_folder), Path(simulation_folder).name + '_taus.pdf'))